In [17]:
import json
import re
import unicodedata
from nltk.corpus import stopwords
import nltk

nltk.download("stopwords")

# Define lecture-related stopwords and patterns to filter out
additional_stop_words = {
    "introduction", "overview", "basics", "principles", "fundamentals", "essentials",
    "concepts", "topics", "outline", "scope", "insights", "lecture", "session",
    "class", "seminar", "discussion", "tutorial", "workshop", "exercise", "module",
    "part", "series", "unit", "goals", "objectives", "summary", "review", "highlights",
    "conclusion", "recap", "continued", "continuation", "advanced", "update", "notes",
    "refresher", "start", "beginning", "end", "final", "first", "second", "third", 
    "perspective", "focus", "study", "methods", "approaches", "applications", "field",
    "understanding", "applications of", "in context", "applied", "introduction to", 
    "explanation", "examining", "exam", "goals", "and", "quiz", "presentation", 
    "presentations", "case", "cases", "bonus", "project", "projects", "problem",
    "problems", "course", "courses", "solution", "solutions", "general", 
    "examples", "example", "lectures", "question", "questions", "answer", "answers",
    "reading", "assignment", "assignments", "definition", "definitions"
}
all_stopwords = set(stopwords.words("english")).union(additional_stop_words)

# Patterns to clean and normalize the titles
patterns_to_remove = r"Lecture\s*\d+:|Lecture\s*\d+|^(\d+[\.\:\;\,\-\—\—]\s*)"
continuation_pattern = r"\b(cont\.?|contd\.?|continued)\b"
parentheses_pattern = r"\(.*?\)"
roman_numeral_pattern = r"\b[IVXLCDM]+\b"
number_pattern = r"\b\d+[\.\:\;\,]?\b"
symbol_cleanup_pattern = r"(?<!\w)[^\w\s/-]|[^\w\s/-](?!\w)"

max_words_per_title = 4

with open('lecture_videos_data.json', 'r') as file:
    stem_data = json.load(file)

# Function to normalize Unicode symbols and replace non-ASCII characters with spaces
def normalize_unicode(text):
    # Replace known Unicode symbols with ASCII equivalents
    text = re.sub(r"[\u2013\u2014\u2015—]", "-", text)  # Normalize dashes to hyphen
    # Normalize and replace non-ASCII characters with a space
    text = unicodedata.normalize("NFKD", text)
    return "".join(c if ord(c) < 128 else " " for c in text)  # Replace non-ASCII with space

# Function to pre-process and clean the title field, including Unicode normalization
def preprocess_title(title):
    # Normalize Unicode symbols and replace non-ASCII characters with spaces
    title = normalize_unicode(title)
    # Remove parentheses content
    title = re.sub(parentheses_pattern, "", title)
    # Remove Roman numerals
    title = re.sub(roman_numeral_pattern, "", title, flags=re.IGNORECASE)
    # Remove standalone numbers and any trailing punctuation
    title = re.sub(number_pattern, "", title)
    # Remove specified patterns and numbers with punctuation
    title = re.sub(patterns_to_remove, "", title, flags=re.IGNORECASE)
    return title.strip()

# Function to filter and separate keywords within title fields, splitting only by commas, colons, and semicolons
def filter_keywords(title):
    title = preprocess_title(title)  # Preprocess the title with Unicode handling
    title_parts = re.split(r'[,:;]', title)  # Split by punctuation to separate distinct concepts
    filtered_parts = []
    for part in title_parts:
        # Remove isolated dots and leading/trailing hyphens within each part
        part = re.sub(r"^\.\s*", "", part).strip()
        part = re.sub(r"^-+|-+$", "", part).strip()  # Remove leading/trailing hyphens
        filtered_words = [
            re.sub(symbol_cleanup_pattern, "", word) for word in part.split()
            if word.lower() not in all_stopwords and not re.search(r"[?!]$", word)
        ]
        # Rejoin words if they are below the word count threshold
        if filtered_words and len(filtered_words) <= max_words_per_title:
            filtered_parts.append(" ".join(filtered_words))
    return filtered_parts

# Function to remove words starting or ending with punctuation
def remove_punctuated_words(title_list):
    cleaned_title_list = [
        word for word in title_list if not re.search(r"^[^\w]|[^\w]$", word)
    ]
    return cleaned_title_list

# Process JSON data to filter out keywords in the title field and exclude continuation entries
unique_lectures = {}
for category, lectures in stem_data.items():
    filtered_lectures = []
    seen_titles = set()
    for lecture in lectures:
        if re.search(continuation_pattern, lecture["title"], re.IGNORECASE):  # Skip continuation markers
            continue
        filtered_title_list = filter_keywords(lecture["title"])  # Filter keywords
        filtered_title_list = remove_punctuated_words(filtered_title_list)  # Remove punctuated words
        if not filtered_title_list:  # Exclude lecture entry if all titles were removed
            continue
        unique_key = (lecture["resource_course_title"], tuple(filtered_title_list))
        if unique_key not in seen_titles:
            lecture["title"] = filtered_title_list
            filtered_lectures.append(lecture)
            seen_titles.add(unique_key)
    unique_lectures[category] = filtered_lectures

with open('filtered_stem_lectures.json', 'w') as outfile:
    json.dump(unique_lectures, outfile, indent=4)
print("Filtered, de-duplicated, and cleaned titles have been saved to 'filtered_stem_lectures.json'")

# Process `resource_course_title` field in the saved filtered data
introductory_phrases = [
    r"Introduction to", r"Principles of", r"Basics of", r"Overview of",
    r"Essentials of", r"Introduction", r"Intro to", r"Introductory", r"Foundations of"
]
intro_pattern = r'\b(?:' + '|'.join(introductory_phrases) + r')\b'

with open('filtered_stem_lectures.json', 'r') as file:
    stem_data = json.load(file)

# Function to clean and extract main subject from `resource_course_title`
def extract_main_subject(title):
    title = re.sub(intro_pattern, '', title, flags=re.IGNORECASE)
    title = re.sub(r'\(.*?\)', '', title)  # Remove course codes or other content in parentheses
    return title.strip()

for category, lectures in stem_data.items():
    for lecture in lectures:
        lecture["resource_course_title"] = extract_main_subject(lecture["resource_course_title"])

with open('filtered_stem_lectures.json', 'w') as outfile:
    json.dump(stem_data, outfile, indent=4)
print("Final cleaned course titles have been saved to 'filtered_stem_lectures.json'")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kroknes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Filtered, de-duplicated, and cleaned titles have been saved to 'filtered_stem_lectures.json'
Final cleaned course titles have been saved to 'filtered_stem_lectures.json'
